In [ ]:
import re
import pandas as pd
import numpy as np

# Read responses (1 row = respondent)
df_resp = pd.read_excel("data/Human Evaluation_ DoR and ORI (135 items + ACE) (respostas).xlsx", dtype=object)
sample = pd.read_csv("out/human_eval_sample.csv")

# Drop timestamp
ts_col = "Carimbo de data/hora"
if ts_col in df_resp.columns:
    df_resp = df_resp.drop(columns=[ts_col])

# Anonymize email column
email_col = "Endereço de e-mail"
if email_col not in df_resp.columns:
    raise KeyError(f"Expected column '{email_col}' in the responses export")

emails = (
    df_resp[email_col]
    .astype(str)
    .str.strip()
    .replace({"": np.nan})
    .dropna()
    .unique()
)
emails = sorted(emails)
if len(emails) > 1000:
    raise ValueError(f"Too many unique emails to map to 3 digits: {len(emails)}")

email_to_id = {e: f"{i:03d}" for i, e in enumerate(emails)}
df_resp = df_resp.rename(columns={email_col: "respondent_id"})
df_resp["respondent_id"] = (
    df_resp["respondent_id"]
    .astype(str)
    .str.strip()
    .map(email_to_id)
)
if df_resp["respondent_id"].isna().any():
    raise ValueError("Some respondents have missing/unknown email values; cannot anonymize reliably")

# Remaining columns are DoR/ORI ratings in pairs
rating_cols = [c for c in df_resp.columns if c != "respondent_id"]
df_long = df_resp.melt(
    id_vars=["respondent_id"],
    value_vars=rating_cols,
    var_name="question",
    value_name="rating",
)

def _parse_item_id(q: str) -> int:
    m = re.search(r"(\d{3})", str(q))
    if not m:
        raise ValueError(f"Could not parse 3-digit item id from: {q}")
    return int(m.group(1))

def _parse_metric(q: str) -> str:
    q = str(q)
    m = re.search(r"\[(DoR|ORI)\]\s*$", q, flags=re.IGNORECASE)
    if m:
        return m.group(1).upper()
    if "DoR" in q:
        return "DOR"
    if "ORI" in q:
        return "ORI"
    raise ValueError(f"Could not detect metric (DoR/ORI) from: {q}")

df_long["item_id"] = df_long["question"].map(_parse_item_id)
df_long["metric"] = df_long["question"].map(_parse_metric)

# Keep as numeric (NaN for blanks)
df_long["rating"] = pd.to_numeric(df_long["rating"], errors="coerce")

# Validate uniqueness per respondent/item/metric
dup = (
    df_long.groupby(["respondent_id", "item_id", "metric"], dropna=False)
    .size()
    .reset_index(name="n")
)
dup = dup[dup["n"] > 1]
if len(dup):
    raise ValueError(f"Duplicate ratings detected:\n{dup.head(20)}")

# Pivot to 1 row per item, 2 cols per respondent: gr_dor_human_XXX, gr_ori_human_XXX
df_wide = df_long.pivot_table(
    index="item_id",
    columns=["respondent_id", "metric"],
    values="rating",
    aggfunc="first",
)

def _col_name(respondent_id: str, metric: str) -> str:
    metric = metric.lower()
    return f"gr_{metric}_human_{respondent_id}"

df_wide.columns = [_col_name(rid, metric) for (rid, metric) in df_wide.columns.to_list()]
df_wide = df_wide.sort_index().reset_index()

# Optional: attach item metadata from the generated sample (Item 001.. = row order in the CSV)
if "sample" in globals():
    df_items = sample.copy()
    df_items["item_id"] = np.arange(1, len(df_items) + 1)
    df_human = df_items.merge(df_wide, on="item_id", how="left")
else:
    df_human = df_wide

print("Respondents:", len(emails))
print("Items:", df_human["item_id"].nunique())
display(df_human.head())


In [ ]:
import numpy as np
import pandas as pd

from scipy.stats import kendalltau, chi2_contingency

if "df_human" not in globals():
    raise NameError("Expected df_human from the previous chunk")

MODEL_PARQUET = "data/tidy_data_2_aed_model.parquet"
df_model = pd.read_parquet(MODEL_PARQUET)

# Join human ratings to model-judge ratings
if "_orig_index" in df_human.columns:
    if "_orig_index" not in df_model.columns:
        df_model = df_model.reset_index().rename(columns={"index": "_orig_index"})
    df_align = df_human.merge(df_model, on="_orig_index", how="left", suffixes=("", "_model"))
elif "item_id" in df_model.columns:
    df_align = df_human.merge(df_model, on="item_id", how="left", suffixes=("", "_model"))
else:
    raise KeyError("Could not join df_human to df_model (need '_orig_index' or 'item_id' in df_model)")


def _cols(prefix: str, df: pd.DataFrame) -> list[str]:
    return [c for c in df.columns if c.startswith(prefix)]


def _discretize_scores(s: pd.Series) -> pd.Series:
    bins = [-np.inf, 1, 3, 5, 7, 9, 10]
    labels = ["[0,1]", "(1,3]", "(3,5]", "(5,7]", "(7,9]", "(9,10]"]
    return pd.cut(pd.to_numeric(s, errors="coerce"), bins=bins, labels=labels, right=True, include_lowest=True)


def _discretize_scores_bins(s: pd.Series, bins: list[float], labels: list[str]) -> pd.Series:
    if len(bins) != len(labels) + 1:
        raise ValueError("bins must have len(labels)+1")
    return pd.cut(
        pd.to_numeric(s, errors="coerce"),
        bins=bins,
        labels=labels,
        right=True,
        include_lowest=True,
    )


# Alternative discretizations for Krippendorff's α (ordinal)
ALPHA_DISCRETIZATION_SCENARIOS = {
    "2_bins_0_5__6_10": {
        "bins": [-np.inf, 5, 10],
        "labels": ["0-5", "6-10"],
    },
    "3_bins_0_3__4_6__7_10": {
        "bins": [-np.inf, 3, 6, 10],
        "labels": ["0-3", "4-6", "7-10"],
    },
    "6_bins_0_1__2_3__3_5__6_7__8_9__9_10": {
        "bins": [-np.inf, 1, 3, 5, 7, 9, 10],
        "labels": ["0-1", "2-3", "3-5", "6-7", "8-9", "9-10"],
    },
}


def _weighted_kappa(a: pd.Series, b: pd.Series, n_cats: int, weights: str = "quadratic") -> float:
    """Cohen's weighted kappa for ordinal categories encoded as 0..n_cats-1 (NaN allowed)."""
    a = pd.to_numeric(a, errors="coerce")
    b = pd.to_numeric(b, errors="coerce")
    m = a.notna() & b.notna()
    if int(m.sum()) == 0:
        return float("nan")

    ai = a[m].astype(int).to_numpy()
    bi = b[m].astype(int).to_numpy()

    k = int(n_cats)
    if k < 2:
        return float("nan")

    O = np.zeros((k, k), dtype=float)
    for x, y in zip(ai, bi):
        if 0 <= x < k and 0 <= y < k:
            O[x, y] += 1.0

    N = O.sum()
    if N == 0:
        return float("nan")

    row = O.sum(axis=1)
    col = O.sum(axis=0)
    E = np.outer(row, col) / N

    idx = np.arange(k)
    if weights == "quadratic":
        W = ((idx[:, None] - idx[None, :]) / (k - 1)) ** 2
    elif weights == "linear":
        W = np.abs(idx[:, None] - idx[None, :]) / (k - 1)
    else:
        raise ValueError("weights must be 'linear' or 'quadratic'")

    num = float((W * O).sum())
    den = float((W * E).sum())
    if den == 0:
        return float("nan")
    return float(1 - (num / den))


def _cramers_v(x: pd.Series, y: pd.Series) -> float:
    tbl = pd.crosstab(x, y)
    if tbl.size == 0:
        return float("nan")
    chi2, _, _, _ = chi2_contingency(tbl)
    n = tbl.to_numpy().sum()
    r, k = tbl.shape
    denom = n * min(r - 1, k - 1)
    return float(np.sqrt(chi2 / denom)) if denom > 0 else float("nan")


def _cramers_v_p(x: pd.Series, y: pd.Series) -> tuple[float, float]:
    tbl = pd.crosstab(x, y)
    if tbl.size == 0:
        return float("nan"), float("nan")
    chi2, p, _, _ = chi2_contingency(tbl)
    n = tbl.to_numpy().sum()
    r, k = tbl.shape
    denom = n * min(r - 1, k - 1)
    v = float(np.sqrt(chi2 / denom)) if denom > 0 else float("nan")
    return v, float(p)


def krippendorff_alpha(values: np.ndarray, level: str = "interval") -> float:
    """Krippendorff's alpha for 2D array (units x raters), NaN = missing."""
    if values.ndim != 2:
        raise ValueError("values must be 2D (units x raters)")

    vals = values.astype(float)
    cats = np.unique(vals[~np.isnan(vals)])
    if cats.size == 0:
        return float("nan")
    cats = np.sort(cats)
    cat_to_i = {c: i for i, c in enumerate(cats)}

    O = np.zeros((len(cats), len(cats)), dtype=float)

    for row in vals:
        row = row[~np.isnan(row)]
        m = len(row)
        if m < 2:
            continue
        u, cnt = np.unique(row, return_counts=True)
        counts = {u_i: int(c_i) for u_i, c_i in zip(u, cnt)}

        # Diagonal
        for c, n_c in counts.items():
            i = cat_to_i[c]
            O[i, i] += n_c * (n_c - 1) / (m - 1)

        # Off-diagonal
        for c, n_c in counts.items():
            i = cat_to_i[c]
            for k, n_k in counts.items():
                if c == k:
                    continue
                j = cat_to_i[k]
                O[i, j] += n_c * n_k / (m - 1)

    n_c = O.sum(axis=1)
    N = n_c.sum()
    if N <= 1:
        return float("nan")

    # Expected coincidences
    E = np.outer(n_c, n_c) / (N - 1)
    np.fill_diagonal(E, n_c * (n_c - 1) / (N - 1))

    # Distances
    if level == "interval":
        D = (cats[:, None] - cats[None, :]) ** 2
    elif level == "ordinal":
        # Krippendorff ordinal distance uses category totals n_c
        D = np.zeros_like(O)
        for i in range(len(cats)):
            for j in range(len(cats)):
                if i == j:
                    continue
                lo, hi = (i, j) if i < j else (j, i)
                s = n_c[lo : hi + 1].sum() - (n_c[lo] + n_c[hi]) / 2
                D[i, j] = s ** 2
    else:
        raise ValueError("level must be 'interval' or 'ordinal'")

    Do = float((O * D).sum() / O.sum()) if O.sum() else float("nan")
    De = float((E * D).sum() / E.sum()) if E.sum() else float("nan")
    if not np.isfinite(Do) or not np.isfinite(De) or De == 0:
        return float("nan")
    return float(1 - (Do / De))


def _summarize(metric: str) -> dict:
    human_cols = _cols(f"gr_{metric}_human_", df_align)
    if not human_cols:
        raise KeyError(f"No human columns found for metric '{metric}'")

    # Model-judge columns: start with gr_{metric}_ but not human and not precomputed medians
    candidates = [
        c for c in _cols(f"gr_{metric}_", df_align)
        if ("human_" not in c)
        and ("outlier" not in c)
        and (not c.endswith("_median"))
    ]
    judge_cols = [
        c for c in candidates
        if not pd.api.types.is_bool_dtype(df_align[c])
    ]
    if not judge_cols:
        raise KeyError(f"No model-judge columns found for metric '{metric}'")

    human_med = df_align[human_cols].median(axis=1, skipna=True)
    judge_med = df_align[judge_cols].median(axis=1, skipna=True)

    valid = human_med.notna() & judge_med.notna()
    try:
        tau, p = kendalltau(human_med[valid], judge_med[valid], variant="b")
    except TypeError:
        tau, p = kendalltau(human_med[valid], judge_med[valid])

    # Krippendorff alpha (inter-rater reliability)
    human_mat = df_align[human_cols].apply(pd.to_numeric, errors="coerce").to_numpy()
    judge_mat = df_align[judge_cols].apply(pd.to_numeric, errors="coerce").to_numpy()
    all_mat = df_align[human_cols + judge_cols].apply(pd.to_numeric, errors="coerce").to_numpy()

    alpha_human_interval = krippendorff_alpha(human_mat, level="interval")
    alpha_judge_interval = krippendorff_alpha(judge_mat, level="interval")
    alpha_all_interval = krippendorff_alpha(all_mat, level="interval")

    # Agreement between median ensembles (humans vs LLM judges) as 2-rater alpha
    ensemble_mat = np.column_stack([
        pd.to_numeric(human_med, errors="coerce").to_numpy(),
        pd.to_numeric(judge_med, errors="coerce").to_numpy(),
    ])
    alpha_ensemble_interval = krippendorff_alpha(ensemble_mat, level="interval")

    # Ordinal alpha on discretized rubric intervals (0..5 ordered)
    human_disc = df_align[human_cols].apply(_discretize_scores).apply(lambda c: c.cat.codes.replace(-1, np.nan))
    judge_disc = df_align[judge_cols].apply(_discretize_scores).apply(lambda c: c.cat.codes.replace(-1, np.nan))
    all_disc = df_align[human_cols + judge_cols].apply(_discretize_scores).apply(lambda c: c.cat.codes.replace(-1, np.nan))
    alpha_human_ordinal = krippendorff_alpha(human_disc.to_numpy(), level="ordinal")
    alpha_judge_ordinal = krippendorff_alpha(judge_disc.to_numpy(), level="ordinal")
    alpha_all_ordinal = krippendorff_alpha(all_disc.to_numpy(), level="ordinal")

    human_med_disc = _discretize_scores(human_med).cat.codes.replace(-1, np.nan)
    judge_med_disc = _discretize_scores(judge_med).cat.codes.replace(-1, np.nan)
    kappa_weighted_rubric = _weighted_kappa(human_med_disc, judge_med_disc, n_cats=6, weights="quadratic")
    alpha_ensemble_ordinal = krippendorff_alpha(
        np.column_stack([human_med_disc.to_numpy(), judge_med_disc.to_numpy()]),
        level="ordinal",
    )

    # Ordinal alpha under alternative discretizations
    alpha_scenarios = {}
    for name, spec in ALPHA_DISCRETIZATION_SCENARIOS.items():
        bins = spec["bins"]
        labels = spec["labels"]
        h_disc = df_align[human_cols].apply(lambda col: _discretize_scores_bins(col, bins=bins, labels=labels))
        j_disc = df_align[judge_cols].apply(lambda col: _discretize_scores_bins(col, bins=bins, labels=labels))
        a_disc = df_align[human_cols + judge_cols].apply(lambda col: _discretize_scores_bins(col, bins=bins, labels=labels))
        h_codes = h_disc.apply(lambda c: c.cat.codes.replace(-1, np.nan))
        j_codes = j_disc.apply(lambda c: c.cat.codes.replace(-1, np.nan))
        a_codes = a_disc.apply(lambda c: c.cat.codes.replace(-1, np.nan))
        alpha_scenarios[f"alpha_human_ordinal__{name}"] = krippendorff_alpha(h_codes.to_numpy(), level="ordinal")
        alpha_scenarios[f"alpha_judge_ordinal__{name}"] = krippendorff_alpha(j_codes.to_numpy(), level="ordinal")
        alpha_scenarios[f"alpha_all_ordinal__{name}"] = krippendorff_alpha(a_codes.to_numpy(), level="ordinal")

        hm = _discretize_scores_bins(human_med, bins=bins, labels=labels).cat.codes.replace(-1, np.nan)
        jm = _discretize_scores_bins(judge_med, bins=bins, labels=labels).cat.codes.replace(-1, np.nan)
        alpha_scenarios[f"kappa_weighted__{name}"] = _weighted_kappa(hm, jm, n_cats=len(labels), weights="quadratic")
        v_s, p_s = _cramers_v_p(
            _discretize_scores_bins(human_med, bins=bins, labels=labels),
            _discretize_scores_bins(judge_med, bins=bins, labels=labels),
        )
        alpha_scenarios[f"cramers_v__{name}"] = float(v_s)
        alpha_scenarios[f"cramers_p__{name}"] = float(p_s)
        alpha_scenarios[f"alpha_ensemble_ordinal__{name}"] = krippendorff_alpha(
            np.column_stack([hm.to_numpy(), jm.to_numpy()]),
            level="ordinal",
        )

    # Cramer's V between discretized medians (with p-value)
    v, v_p = _cramers_v_p(_discretize_scores(human_med), _discretize_scores(judge_med))

    return {
        "metric": metric.upper(),
        "n_items": int(valid.sum()),
        "kendall_tau_b": float(tau),
        "kendall_p": float(p),
        "alpha_human_interval": float(alpha_human_interval),
        "alpha_judge_interval": float(alpha_judge_interval),
        "alpha_all_interval": float(alpha_all_interval),
        "alpha_ensemble_interval": float(alpha_ensemble_interval),
        "alpha_human_ordinal": float(alpha_human_ordinal),
        "alpha_judge_ordinal": float(alpha_judge_ordinal),
        "alpha_all_ordinal": float(alpha_all_ordinal),
        "alpha_ensemble_ordinal": float(alpha_ensemble_ordinal),
        "kappa_weighted_rubric": float(kappa_weighted_rubric),
        **{k: float(v) for k, v in alpha_scenarios.items()},
        "cramers_v_discretized_medians": float(v),
        "cramers_p_discretized_medians": float(v_p),
        "n_humans": len(human_cols),
        "n_model_judges": len(judge_cols),
    }


summary = pd.DataFrame([_summarize("dor"), _summarize("ori")])

# Structured α table for printing
def _alpha_row(metric_row: pd.Series, scale: str, kind: str, suffix: str = "") -> dict:
    suf = f"__{suffix}" if suffix else ""
    if kind == "interval":
        return {
            "metric": metric_row["metric"],
            "kind": "interval",
            "scale": scale,
            "alpha_humans": metric_row["alpha_human_interval"],
            "alpha_llms": metric_row["alpha_judge_interval"],
            "alpha_all": metric_row["alpha_all_interval"],
            "alpha_ensemble_medians": metric_row["alpha_ensemble_interval"],
            "kappa_weighted": np.nan,
            "cramers_v": np.nan,
            "cramers_p": np.nan,
        }

    # ordinal
    return {
        "metric": metric_row["metric"],
        "kind": "ordinal",
        "scale": scale,
        "alpha_humans": metric_row[f"alpha_human_ordinal{suf}"] if suf else metric_row["alpha_human_ordinal"],
        "alpha_llms": metric_row[f"alpha_judge_ordinal{suf}"] if suf else metric_row["alpha_judge_ordinal"],
        "alpha_all": metric_row[f"alpha_all_ordinal{suf}"] if suf else metric_row["alpha_all_ordinal"],
        "alpha_ensemble_medians": metric_row[f"alpha_ensemble_ordinal{suf}"] if suf else metric_row["alpha_ensemble_ordinal"],
        "kappa_weighted": metric_row[f"kappa_weighted{suf}"] if suf else metric_row["kappa_weighted_rubric"],
        "cramers_v": metric_row[f"cramers_v{suf}"] if suf else metric_row["cramers_v_discretized_medians"],
        "cramers_p": metric_row[f"cramers_p{suf}"] if suf else metric_row["cramers_p_discretized_medians"],
    }


alpha_rows = []
for _, r in summary.iterrows():
    alpha_rows.append(_alpha_row(r, scale="raw_0_10", kind="interval"))
    alpha_rows.append(_alpha_row(r, scale="rubric_6_bins", kind="ordinal"))
    for name in ALPHA_DISCRETIZATION_SCENARIOS.keys():
        alpha_rows.append(_alpha_row(r, scale=name, kind="ordinal", suffix=name))

alpha_table = pd.DataFrame(alpha_rows)

# Order rows: interval first, then ordinal (rubric first, then scenarios)
scale_order = ["raw_0_10", "rubric_6_bins"] + list(ALPHA_DISCRETIZATION_SCENARIOS.keys())
alpha_table["_scale_order"] = alpha_table["scale"].map({s: i for i, s in enumerate(scale_order)})
alpha_table["_kind_order"] = alpha_table["kind"].map({"interval": 0, "ordinal": 1})
alpha_table = alpha_table.sort_values(["metric", "_kind_order", "_scale_order"]).drop(columns=["_kind_order", "_scale_order"]) 

# Pretty print
alpha_table_fmt = alpha_table.copy()
for c in ["alpha_humans", "alpha_llms", "alpha_all", "alpha_ensemble_medians", "kappa_weighted", "cramers_v"]:
    alpha_table_fmt[c] = pd.to_numeric(alpha_table_fmt[c], errors="coerce").round(3)
alpha_table_fmt["cramers_p"] = pd.to_numeric(alpha_table_fmt["cramers_p"], errors="coerce")

print("=")
print("Agreement summary — Krippendorff’s α, weighted κ (bins), Cramér’s V (+p)")
print(alpha_table_fmt.to_string(index=False))

# -----------------------------
# Kendall τ_b table (humans vs LLMs)
# -----------------------------

def _kendall_tau_b(a: pd.Series, b: pd.Series) -> float:
    a = pd.to_numeric(a, errors="coerce")
    b = pd.to_numeric(b, errors="coerce")
    m = a.notna() & b.notna()
    if int(m.sum()) < 3:
        return float("nan")
    try:
        tau, _ = kendalltau(a[m], b[m], variant="b")
    except TypeError:
        tau, _ = kendalltau(a[m], b[m])
    return float(tau)


def _pairwise_tau(df: pd.DataFrame, cols: list[str]) -> pd.Series:
    taus = []
    for i in range(len(cols)):
        for j in range(i + 1, len(cols)):
            taus.append(_kendall_tau_b(df[cols[i]], df[cols[j]]))
    return pd.Series(taus, dtype=float)


def _cross_tau(df: pd.DataFrame, a_cols: list[str], b_cols: list[str]) -> pd.Series:
    taus = []
    for a in a_cols:
        for b in b_cols:
            taus.append(_kendall_tau_b(df[a], df[b]))
    return pd.Series(taus, dtype=float)


def _tau_summary(taus: pd.Series) -> dict:
    taus = pd.to_numeric(taus, errors="coerce").dropna()
    if len(taus) == 0:
        return {"n_pairs": 0, "tau_median": np.nan, "tau_q25": np.nan, "tau_q75": np.nan}
    return {
        "n_pairs": int(len(taus)),
        "tau_median": float(taus.median()),
        "tau_q25": float(taus.quantile(0.25)),
        "tau_q75": float(taus.quantile(0.75)),
    }


tau_rows = []
for metric in ["dor", "ori"]:
    human_cols = [c for c in df_align.columns if c.startswith(f"gr_{metric}_human_")]
    llm_cols = [
        c
        for c in df_align.columns
        if c.startswith(f"gr_{metric}_")
        and ("human_" not in c)
        and ("outlier" not in c)
        and (not c.endswith("_median"))
        and (not pd.api.types.is_bool_dtype(df_align[c]))
    ]

    s_h = _tau_summary(_pairwise_tau(df_align, human_cols))
    s_l = _tau_summary(_pairwise_tau(df_align, llm_cols))
    s_x = _tau_summary(_cross_tau(df_align, human_cols, llm_cols))

    human_med = df_align[human_cols].apply(pd.to_numeric, errors="coerce").median(axis=1, skipna=True)
    llm_med = df_align[llm_cols].apply(pd.to_numeric, errors="coerce").median(axis=1, skipna=True)
    tau_med = _kendall_tau_b(human_med, llm_med)

    tau_rows.append({"metric": metric.upper(), "comparison": "humans_within", **s_h})
    tau_rows.append({"metric": metric.upper(), "comparison": "llms_within", **s_l})
    tau_rows.append({"metric": metric.upper(), "comparison": "humans_vs_llms_pairs", **s_x})
    tau_rows.append({
        "metric": metric.upper(),
        "comparison": "ensemble_medians_human_vs_llm",
        "n_pairs": 1,
        "tau_median": float(tau_med),
        "tau_q25": np.nan,
        "tau_q75": np.nan,
    })


tau_table = pd.DataFrame(tau_rows)
for c in ["tau_median", "tau_q25", "tau_q75"]:
    tau_table[c] = pd.to_numeric(tau_table[c], errors="coerce").round(3)

print("=")
print("Kendall τ_b summary")
print(tau_table.to_string(index=False))


In [ ]:
import warnings
import numpy as np
import pandas as pd

from scipy.stats import kendalltau

# Suppress noisy warnings (statsmodels/pandas)
warnings.filterwarnings("ignore")
try:
    from statsmodels.tools.sm_exceptions import ConvergenceWarning
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
except Exception:
    pass

# Methodological diagnostics (same design: 135 items, 3 humans, 5 judges)
# 1) Pairwise (human h, judge j) association distribution
# 2) Ordinal model with rater severity (fixed-effects OrderedLogit)
# 3) Per-rater standardization before aggregation

if "df_align" not in globals():
    raise NameError("Expected df_align from the previous chunk")


def _cols(prefix: str, df: pd.DataFrame) -> list[str]:
    return [c for c in df.columns if c.startswith(prefix)]


def _kendall_tau_b(a: pd.Series, b: pd.Series):
    a = pd.to_numeric(a, errors="coerce")
    b = pd.to_numeric(b, errors="coerce")
    m = a.notna() & b.notna()
    if int(m.sum()) < 3:
        return float("nan"), float("nan"), int(m.sum())
    try:
        tau, p = kendalltau(a[m], b[m], variant="b")
    except TypeError:
        tau, p = kendalltau(a[m], b[m])
    return float(tau), float(p), int(m.sum())


def _discretize_scores(s: pd.Series) -> pd.Series:
    bins = [-np.inf, 1, 3, 5, 7, 9, 10]
    labels = ["[0,1]", "(1,3]", "(3,5]", "(5,7]", "(7,9]", "(9,10]"]
    return pd.cut(pd.to_numeric(s, errors="coerce"), bins=bins, labels=labels, right=True, include_lowest=True)


def _summary_iqr(x: pd.Series) -> dict:
    x = pd.to_numeric(x, errors="coerce")
    return {
        "median": float(x.median()),
        "q25": float(x.quantile(0.25)),
        "q75": float(x.quantile(0.75)),
    }


def pairwise_association(metric: str) -> pd.DataFrame:
    human_cols = _cols(f"gr_{metric}_human_", df_align)
    judge_cols = [
        c
        for c in _cols(f"gr_{metric}_", df_align)
        if ("human_" not in c)
        and ("outlier" not in c)
        and (not c.endswith("_median"))
        and (not pd.api.types.is_bool_dtype(df_align[c]))
    ]
    if not human_cols or not judge_cols:
        raise KeyError(f"Missing columns for metric={metric}")

    rows = []
    for h in human_cols:
        for j in judge_cols:
            tau, p, n = _kendall_tau_b(df_align[h], df_align[j])
            rows.append({
                "metric": metric.upper(),
                "human": h,
                "judge": j,
                "tau_b": tau,
                "p": p,
                "n": n,
            })
    return pd.DataFrame(rows)


def standardization_analysis(metric: str) -> dict:
    human_cols = _cols(f"gr_{metric}_human_", df_align)
    judge_cols = [
        c
        for c in _cols(f"gr_{metric}_", df_align)
        if ("human_" not in c)
        and ("outlier" not in c)
        and (not c.endswith("_median"))
        and (not pd.api.types.is_bool_dtype(df_align[c]))
    ]

    h = df_align[human_cols].apply(pd.to_numeric, errors="coerce")
    j = df_align[judge_cols].apply(pd.to_numeric, errors="coerce")

    h_med = h.median(axis=1, skipna=True)
    j_med = j.median(axis=1, skipna=True)
    tau_raw, _, n_raw = _kendall_tau_b(h_med, j_med)

    def zscore(col: pd.Series) -> pd.Series:
        mu = col.mean(skipna=True)
        sd = col.std(skipna=True, ddof=0)
        if not np.isfinite(sd) or sd == 0:
            return col * np.nan
        return (col - mu) / sd

    h_z = h.apply(zscore)
    j_z = j.apply(zscore)
    tau_z, _, n_z = _kendall_tau_b(h_z.median(axis=1, skipna=True), j_z.median(axis=1, skipna=True))

    h_r = h.rank(axis=0, method="average", na_option="keep")
    j_r = j.rank(axis=0, method="average", na_option="keep")
    tau_rank, _, n_rank = _kendall_tau_b(h_r.median(axis=1, skipna=True), j_r.median(axis=1, skipna=True))

    return {
        "metric": metric.upper(),
        "tau_raw": tau_raw,
        "n_raw": n_raw,
        "tau_zscore": tau_z,
        "n_zscore": n_z,
        "tau_rank": tau_rank,
        "n_rank": n_rank,
    }


def ordinal_severity_model(metric: str):
    """Fixed-effects OrderedLogit: score_bin ~ item + rater.
    This is not fully hierarchical shrinkage, but separates item signal vs rater severity.
    """
    try:
        from statsmodels.miscmodels.ordinal_model import OrderedModel
    except Exception as e:
        print(f"[SKIP] OrderedLogit needs statsmodels: {type(e).__name__}")
        return None

    def to_long(cols: list[str], group: str) -> pd.DataFrame:
        parts = []
        for c in cols:
            parts.append(pd.DataFrame({
                "item_id": df_align["item_id"] if "item_id" in df_align.columns else np.arange(1, len(df_align) + 1),
                "rater": c,
                "group": group,
                "score": pd.to_numeric(df_align[c], errors="coerce"),
            }))
        return pd.concat(parts, ignore_index=True)

    human_cols = _cols(f"gr_{metric}_human_", df_align)
    judge_cols = [
        c
        for c in _cols(f"gr_{metric}_", df_align)
        if ("human_" not in c)
        and ("outlier" not in c)
        and (not c.endswith("_median"))
        and (not pd.api.types.is_bool_dtype(df_align[c]))
    ]

    long_h = to_long(human_cols, "human")
    long_j = to_long(judge_cols, "judge")

    def fit_group(long_df: pd.DataFrame):
        y_cat = _discretize_scores(long_df["score"]).cat.codes.replace(-1, np.nan)
        m = y_cat.notna()
        y = y_cat[m].astype(int)

        X = pd.get_dummies(long_df.loc[m, ["item_id", "rater"]].astype({"item_id": str, "rater": str}), drop_first=True)
        model = OrderedModel(y, X, distr="logit")
        res = model.fit(method="bfgs", disp=False, maxiter=200)

        # Extract item effects (baseline item = 0)
        item_terms = [c for c in X.columns if c.startswith("item_id_")]
        rater_terms = [c for c in X.columns if c.startswith("rater_")]

        items = sorted(long_df["item_id"].dropna().unique())
        theta = {items[0]: 0.0}
        for it in items[1:]:
            key = f"item_id_{it}"
            theta[it] = float(res.params.get(key, 0.0))

        raters = sorted(long_df["rater"].dropna().unique())
        b = {raters[0]: 0.0}
        for r in raters[1:]:
            key = f"rater_{r}"
            b[r] = float(res.params.get(key, 0.0))

        return {
            "res": res,
            "theta": pd.Series(theta).sort_index(),
            "severity": pd.Series(b),
            "n_obs": int(m.sum()),
            "n_items": int(len(items)),
            "n_raters": int(len(raters)),
        }

    out_h = fit_group(long_h)
    out_j = fit_group(long_j)

    # Compare item latent effects across groups
    common = out_h["theta"].index.intersection(out_j["theta"].index)
    tau_theta, _, _ = _kendall_tau_b(out_h["theta"].loc[common], out_j["theta"].loc[common])

    return {
        "metric": metric.upper(),
        "human": out_h,
        "judge": out_j,
        "tau_theta_human_vs_judge": tau_theta,
        "severity_human": out_h["severity"],
        "severity_judge": out_j["severity"],
    }


def _human_cols(metric: str) -> list[str]:
    return [c for c in df_align.columns if c.startswith(f"gr_{metric}_human_")]


def _llm_cols(metric: str) -> list[str]:
    return [
        c
        for c in _cols(f"gr_{metric}_", df_align)
        if ("human_" not in c)
        and ("outlier" not in c)
        and (not c.endswith("_median"))
        and (not pd.api.types.is_bool_dtype(df_align[c]))
    ]


def _pairwise_within(cols: list[str], label: str, metric: str) -> pd.DataFrame:
    rows = []
    for i in range(len(cols)):
        for j in range(i + 1, len(cols)):
            tau, p, n = _kendall_tau_b(df_align[cols[i]], df_align[cols[j]])
            rows.append({
                "metric": metric.upper(),
                "group": label,
                "rater_a": cols[i],
                "rater_b": cols[j],
                "tau_b": tau,
                "n": n,
            })
    return pd.DataFrame(rows)


def _print_within(df_pairs: pd.DataFrame, title: str) -> None:
    if df_pairs.empty:
        print(f"{title}: (no pairs)")
        return
    s_all = _summary_iqr(df_pairs["tau_b"])
    print(title)
    print(f"pairs={len(df_pairs)} tau_b median={s_all['median']:.3f} IQR=[{s_all['q25']:.3f}, {s_all['q75']:.3f}]")
    show = df_pairs.sort_values("tau_b")[["rater_a", "rater_b", "tau_b", "n"]]
    print(show.to_string(index=False))


# Pre-fit ordinal models once (used in all sections)
ord_dor = ordinal_severity_model("dor")
ord_ori = ordinal_severity_model("ori")

def _print_ordinal_group(obj: dict | None, group: str) -> None:
    if obj is None:
        print("[SKIP] OrderedLogit FE not available")
        return
    out = obj[group]
    sev = out["severity"].sort_values()
    print(f"n_obs={out['n_obs']} n_items={out['n_items']} n_raters={out['n_raters']}")
    print("severity (lowest/highest):")
    print(pd.concat([sev.head(3), sev.tail(3)]).to_string())


def _print_ordinal_ensemble(obj: dict | None) -> None:
    if obj is None:
        return
    th = obj["human"]["theta"]
    tj = obj["judge"]["theta"]
    common = th.index.intersection(tj.index)
    diff = (tj.loc[common] - th.loc[common]).astype(float)
    print(f"tau(theta_humans vs theta_llms) = {obj['tau_theta_human_vs_judge']:.3f}")
    print(f"theta bias (llm - human): median={diff.median():.3f} IQR=[{diff.quantile(0.25):.3f}, {diff.quantile(0.75):.3f}]")


# -----------------------------
# HUMANS
# -----------------------------
print("=")
print("Methodological diagnostics — HUMANS")
for metric in ["dor", "ori"]:
    cols = _human_cols(metric)
    df_pairs = _pairwise_within(cols, label="humans", metric=metric)
    _print_within(df_pairs, f"Within-humans τ_b ({metric.upper()})")

print("-")
print("Ordinal model (humans):")
_print_ordinal_group(ord_dor, "human")
_print_ordinal_group(ord_ori, "human")


# -----------------------------
# LLMs
# -----------------------------
print("=")
print("Methodological diagnostics — LLMs")
for metric in ["dor", "ori"]:
    cols = _llm_cols(metric)
    df_pairs = _pairwise_within(cols, label="llms", metric=metric)
    _print_within(df_pairs, f"Within-LLMs τ_b ({metric.upper()})")

print("-")
print("Ordinal model (LLMs):")
_print_ordinal_group(ord_dor, "judge")
_print_ordinal_group(ord_ori, "judge")


# -----------------------------
# ENSEMBLE MEDIANS (humans vs LLMs)
# -----------------------------
print("=")
print("Methodological diagnostics — Ensemble medians (humans vs LLMs)")

# 1) Cross-pairs (human × llm) distribution and judge ranking
pair_dor = pairwise_association("dor")
pair_ori = pairwise_association("ori")

def _cross_print(df_pairs: pd.DataFrame, title: str):
    s_all = _summary_iqr(df_pairs["tau_b"])
    by_j = df_pairs.groupby("judge")["tau_b"].median().sort_values()
    print(title)
    print(f"pairs={len(df_pairs)} tau_b median={s_all['median']:.3f} IQR=[{s_all['q25']:.3f}, {s_all['q75']:.3f}]")
    print("lowest LLM judges (median over humans):")
    print(by_j.head(5).to_string())
    print("highest LLM judges (median over humans):")
    print(by_j.tail(5).to_string())


_cross_print(pair_dor, "Cross human×LLM τ_b (DoR)")
_cross_print(pair_ori, "Cross human×LLM τ_b (ORI)")

print("-")
print("Ordinal model (ensemble comparison):")
_print_ordinal_ensemble(ord_dor)
_print_ordinal_ensemble(ord_ori)

# 3) Per-rater standardization before aggregation
std_dor = standardization_analysis("dor")
std_ori = standardization_analysis("ori")
print("-")
print("Standardization before aggregation (median humans vs median LLMs)")
for obj in [std_dor, std_ori]:
    print(
        f"{obj['metric']}: tau_raw={obj['tau_raw']:.3f} (n={obj['n_raw']}) | "
        f"tau_zscore={obj['tau_zscore']:.3f} (n={obj['n_zscore']}) | "
        f"tau_rank={obj['tau_rank']:.3f} (n={obj['n_rank']})"
    )


In [ ]:
import numpy as np
import pandas as pd

# Where humans and LLM-judges agree/disagree the most

if "df_align" not in globals():
    # Try to construct df_align from df_human + model parquet (so this chunk can run standalone)
    if "df_human" not in globals():
        raise NameError(
            "Expected df_align (or df_human) from previous chunks. "
            "Run the chunks that create df_human (responses import) and df_align (merge with model parquet)."
        )

    MODEL_PARQUET = "data/tidy_data_2_aed_model.parquet"
    try:
        df_model = pd.read_parquet(MODEL_PARQUET)
    except Exception as e:
        raise RuntimeError(
            f"Could not read {MODEL_PARQUET}. Ensure parquet support is installed (pyarrow or fastparquet). "
            f"Original error: {type(e).__name__}: {e}"
        )

    if "_orig_index" in df_human.columns:
        if "_orig_index" not in df_model.columns:
            df_model = df_model.reset_index().rename(columns={"index": "_orig_index"})
        df_align = df_human.merge(df_model, on="_orig_index", how="left", suffixes=("", "_model"))
    elif "item_id" in df_model.columns and "item_id" in df_human.columns:
        df_align = df_human.merge(df_model, on="item_id", how="left", suffixes=("", "_model"))
    else:
        raise KeyError(
            "Could not join df_human to df_model. Need either '_orig_index' in df_human, or 'item_id' in both."
        )


def _discretize_rubric(s: pd.Series) -> pd.Series:
    bins = [-np.inf, 1, 3, 5, 7, 9, 10]
    labels = ["[0,1]", "(1,3]", "(3,5]", "(5,7]", "(7,9]", "(9,10]"]
    return pd.cut(pd.to_numeric(s, errors="coerce"), bins=bins, labels=labels, right=True, include_lowest=True)


def _human_cols(metric: str) -> list[str]:
    return [c for c in df_align.columns if c.startswith(f"gr_{metric}_human_")]


def _llm_cols(metric: str) -> list[str]:
    return [
        c
        for c in df_align.columns
        if c.startswith(f"gr_{metric}_")
        and ("human_" not in c)
        and ("outlier" not in c)
        and (not c.endswith("_median"))
        and (not pd.api.types.is_bool_dtype(df_align[c]))
    ]


META_COLS = [c for c in ["item_id", "_orig_index", "source", "model", "prompt_type", "item", "answer"] if c in df_align.columns]


def _item_agreement(metric: str) -> pd.DataFrame:
    h_cols = _human_cols(metric)
    l_cols = _llm_cols(metric)
    if not h_cols or not l_cols:
        raise KeyError(f"Missing columns for metric={metric}")

    h = df_align[h_cols].apply(pd.to_numeric, errors="coerce")
    l = df_align[l_cols].apply(pd.to_numeric, errors="coerce")

    h_med = h.median(axis=1, skipna=True)
    l_med = l.median(axis=1, skipna=True)

    out = df_align[META_COLS].copy() if META_COLS else pd.DataFrame(index=df_align.index)
    out[f"{metric}_human_median"] = h_med
    out[f"{metric}_llm_median"] = l_med
    out[f"{metric}_abs_diff"] = (h_med - l_med).abs()

    out[f"{metric}_human_iqr"] = h.quantile(0.75, axis=1) - h.quantile(0.25, axis=1)
    out[f"{metric}_llm_iqr"] = l.quantile(0.75, axis=1) - l.quantile(0.25, axis=1)

    hb = _discretize_rubric(h_med)
    lb = _discretize_rubric(l_med)
    out[f"{metric}_human_bin"] = hb.astype(str)
    out[f"{metric}_llm_bin"] = lb.astype(str)
    out[f"{metric}_bin_match"] = (hb == lb)

    return out


dor_items = _item_agreement("dor")
ori_items = _item_agreement("ori")

# Combine for an overall view
combo = dor_items.merge(
    ori_items[[c for c in ori_items.columns if c not in META_COLS]],
    left_index=True,
    right_index=True,
    how="inner",
)
combo["abs_diff_mean_dor_ori"] = combo[["dor_abs_diff", "ori_abs_diff"]].mean(axis=1)
combo["both_bins_match"] = combo["dor_bin_match"] & combo["ori_bin_match"]

def _print_top(df: pd.DataFrame, title: str, sort_col: str, ascending: bool, n: int = 10):
    cols = META_COLS + [
        "dor_human_median",
        "dor_llm_median",
        "dor_abs_diff",
        "dor_human_bin",
        "dor_llm_bin",
        "dor_bin_match",
        "ori_human_median",
        "ori_llm_median",
        "ori_abs_diff",
        "ori_human_bin",
        "ori_llm_bin",
        "ori_bin_match",
        "abs_diff_mean_dor_ori",
        "both_bins_match",
    ]
    cols = [c for c in cols if c in df.columns]
    view = df.sort_values(sort_col, ascending=ascending).head(n)[cols].copy()
    for c in ["dor_human_median","dor_llm_median","dor_abs_diff","ori_human_median","ori_llm_median","ori_abs_diff","abs_diff_mean_dor_ori"]:
        if c in view.columns:
            view[c] = pd.to_numeric(view[c], errors="coerce").round(3)
    print("=")
    print(title)
    print(view.to_string(index=False))


_print_top(combo, "Most agreement (lowest mean |Δ| across DoR/ORI)", "abs_diff_mean_dor_ori", ascending=True)
_print_top(combo, "Most disagreement (highest mean |Δ| across DoR/ORI)", "abs_diff_mean_dor_ori", ascending=False)

# Trait-specific views
_print_top(combo, "Most agreement (DoR)", "dor_abs_diff", ascending=True)
_print_top(combo, "Most disagreement (DoR)", "dor_abs_diff", ascending=False)
_print_top(combo, "Most agreement (ORI)", "ori_abs_diff", ascending=True)
_print_top(combo, "Most disagreement (ORI)", "ori_abs_diff", ascending=False)

# Quick counts
print("=")
print("Bin agreement rates (rubric bins)")
print(pd.DataFrame({
    "metric": ["DoR", "ORI", "both"],
    "bin_match_rate": [
        float(combo["dor_bin_match"].mean()),
        float(combo["ori_bin_match"].mean()),
        float(combo["both_bins_match"].mean()),
    ],
}).assign(bin_match_rate=lambda d: d["bin_match_rate"].round(3)).to_string(index=False))
